# Before Analysis...

In [1]:
import numpy as np
import mne
import sklearn
import matplotlib.pyplot as plt
from scipy import stats
from decimal import Decimal, getcontext
import pandas as pd
import ast

# 분석대상 이름 입력
name = input("분석대상 이름을 입력하세요.")
purpose = input("목적을 입력하세요. TFR or loading")

if purpose == "TFR":
    print(f"TFR을 선택하였습니다.")
    # fif 파일 불러오기
    file_path = rf'H:\Mg_EEG\edf_files\{name}_7200_clean.fif'
    clean_data = mne.io.read_raw_fif(file_path, preload=True)

    print(f"fif 파일을 성공적으로 불러왔습니다: {file_path}")
    
    # Computing TFR
    print(f"TFR을 계산 중 입니다.")
    tfr = clean_data.compute_tfr(method='multitaper', freqs=np.arange(1, 31), tmin=0, tmax=7199, n_jobs=-1, reject_by_annotation=False)
    print(f"TFR 계산이 완료되었습니다.")

    # TFR 파일 저장
    saving_path = rf'H:\Mg_EEG\tfr_files\{name}_7200_tfr.h5'
    tfr.save(saving_path, overwrite=True)
    print(f"TFR 파일이 성공적으로 저장되었습니다.")

    # Saving raw file for back-up
    tfr_raw = tfr.copy()

elif purpose == "loading":
    print(f"loading을 선택하였습니다.")
    # Loading saved TFR file
    file_path = rf'H:\Mg_EEG\tfr_files\{name}_7200_tfr.h5'
    tfr = mne.time_frequency.read_tfrs(file_path)
    print(f"성공적으로 tfr 파일을 로딩하였습니다.")

    # Saving raw file for back-up
    tfr_raw = tfr.copy()

else:
    print("올바른 값을 입력하시오")
    quit()

def tfr_mean(tfr):
    mean_electrode = np.mean(tfr.data, axis=0) # Averaging across electrode
    mean_frequency = np.mean(mean_electrode, axis=0) # Averaging across frequency
    return(mean_frequency)

loading을 선택하였습니다.
Reading H:\Mg_EEG\tfr_files\김충연_7200_tfr.h5 ...
성공적으로 tfr 파일을 로딩하였습니다.


In [ ]:
#raw fif file 보고 싶다면 사용
#clean_data.plot()

In [ ]:
#데이터가 뭔가 이상할 때 리셋하는 복구 코드
#tfr = tfr_raw

In [5]:
# large artifact data load
csv_file_path = r'C:\Users\esin4\OneDrive\바탕 화면\Github\Mg_infusion_coma\large_artifact.csv'

large_artifact_data = pd.read_csv(csv_file_path, encoding='utf-8-sig')

# 입력한 이름이 데이터프레임에 존재하는지 확인하고 좌표 가져오기
if name in large_artifact_data['Name'].values:
    # 입력한 이름에 해당하는 데이터 가져오기
    coordinates = large_artifact_data[large_artifact_data['Name'] == name]['Coordinates'].values[0]
    
    # 좌표가 'skip'이면 빈 리스트로 설정
    if coordinates == 'skip':
        large_artifact = []
    else:
        # 좌표 문자열을 리스트로 변환
        import ast
        large_artifact = ast.literal_eval(coordinates)
else:
    print(f"{input_name}이(가) 데이터에 없습니다.")
    large_artifact = []

# 결과 출력
print("large_artifact 리스트:")
print(large_artifact)

large_artifact 리스트:
[(642, 669), (689, 691), (711, 713), (2488, 2490), (2574, 2578), (2651, 2654), (2658, 2661), (2662, 2663), (2666, 2679), (2692, 2694), (2701, 2703), (2718, 2720), (4014, 4018), (4039, 4041), (4053, 4055), (4059, 4061), (4070, 4078), (5075, 5077), (6935, 6937), (6942, 6955), (6960, 7000), (7139, 7144)]


# independent t-test

In [6]:
# Before vs After segmentation
tfr_before = tfr.copy().crop(tmin=0, tmax=3600, include_tmax=False)
tfr_after = tfr.copy().crop(tmin=3600, tmax=7199, include_tmax=False)

# Averaging
before_mean = tfr_mean(tfr_before)
after_mean = tfr_mean(tfr_after)

# Indexing about Large artifact
before_idx = []
after_idx = []

for t_start, t_end in large_artifact:
    if t_start < 3600 and t_end < 3600:
        before_idx.extend(range(t_start*200, t_end*200))
    elif t_start < 3600 and t_end >= 3600:
        before_idx.extend(range(t_start*200, 3600*200))
        after_idx.extend(range(0, (t_end-3600)*200))
    else:
        after_idx.extend(range((t_start-3600)*200, (t_end-3600)*200))

# Removing Large artifacts
before_mean_clean = np.delete(before_mean, before_idx)
after_mean_clean = np.delete(after_mean, after_idx)

# independent t-test
# precision
getcontext().prec = 50

print(f"{name}, independent t test")
# mean of before vs after
print("mean_before : \n", np.mean(before_mean_clean))
print("mean_after : \n", np.mean(after_mean_clean))

# t-test
t_stat, p_value = stats.ttest_ind(before_mean_clean, after_mean_clean)

# print to decimal object
p_value_decimal = Decimal(p_value)
print ("t_statistics : \n", t_stat)
print(f"p-value: \n {p_value_decimal:.100f}")

# Define Cohens'D 
def cohens_d(group1, group2):
    # Calculate mean, sd
    mean1, mean2 = np.mean(group1), np.mean(group2)
    std1, std2 = np.std(group1, ddof=1), np.std(group2, ddof=1)
    
    # sample size
    n1, n2 = len(group1), len(group2)
    
    # Calculate pooled sd
    pooled_std = np.sqrt(((n1 - 1) * std1**2 + (n2 - 1) * std2**2) / (n1 + n2 - 2))
    
    # Cohen's d
    d = (mean1 - mean2) / pooled_std
    return d

# Calculate Effect Size
print("effect size \n", cohens_d(before_mean, after_mean))

김충연, independent t test
mean_before : 
 1.102976585078232e-09
mean_after : 
 3.3466908487452605e-09
t_statistics : 
 -27.76956297940934
p-value: 
 0.0000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000
effect size 
 -0.05201302852932555


# Paired t-test

In [7]:
# Before vs After segmentation
tfr_before = tfr.copy().crop(tmin=0, tmax=3600, include_tmax=False)
tfr_after = tfr.copy().crop(tmin=3600, tmax=7199, include_tmax=False)

# Averaging
before_mean = tfr_mean(tfr_before)
after_mean = tfr_mean(tfr_after)

# trim the data 무조건 7200초 맞춰서 이 부분 안 써도 되도록 해야함. 
if len(before_mean) > len(after_mean):
    before_mean = before_mean[:len(after_mean)]
elif len(before_mean) < len(after_mean):
    after_mean = after_mean[:len(before_mean)]
else:
    quit

len(before_mean) == len(after_mean)

# Indexing about large artifact

before_idx = set()
after_idx = set()

for t_start, t_end in large_artifact:
    if t_start < 3600 and t_end < 3600:
        before_idx.update(range(t_start*200, t_end*200))
        after_idx.update(range(t_start*200, t_end*200))
    elif t_start < 3600 and t_end >= 3600:
        before_idx.update(range(t_start*200, 3600*200))
        after_idx.update(range(t_start*200, 3600*200))
        after_idx.update(range(0, (t_end-3600)*200))
        before_idx.update(range(0, (t_end-3600)*200))
    else:
        after_idx.update(range((t_start-3600)*200, (t_end-3600)*200))
        before_idx.update(range((t_start-3600)*200, (t_end-3600)*200))

# Convert sets back to sorted lists
before_idx = sorted(before_idx)
after_idx = sorted(after_idx)

before_mean_clean = np.delete(before_mean, before_idx)
after_mean_clean = np.delete(after_mean, after_idx)

# Paired t-test
# precision
getcontext().prec = 50

print(f"{name}, paired t test")
# mean of before vs after
print("mean_before :\n ", np.mean(before_mean_clean))
print("mean_after : \n", np.mean(after_mean_clean))

# Kolmogorov-Smirnov test
difference = before_mean_clean-after_mean_clean
statistic, p_value = stats.kstest(difference, stats.norm.cdf)
print(f"KS Statistic: \n {statistic}")
print(f"KS p-value: \n {p_value}")

# t-test
t_stat, p_value = stats.ttest_rel(before_mean, after_mean)

# print to decimal object
p_value_decimal = Decimal(p_value)
print ("t_statistics : \n", t_stat)
print(f"p-value: \n {p_value_decimal:.30f}")

# Define Cohens'D 
def cohens_d(group1, group2):
    # Calculate mean, sd
    mean1, mean2 = np.mean(group1), np.mean(group2)
    std1, std2 = np.std(group1, ddof=1), np.std(group2, ddof=1)
    
    # sample size
    n1, n2 = len(group1), len(group2)
    
    # Calculate pooled sd
    pooled_std = np.sqrt(((n1 - 1) * std1**2 + (n2 - 1) * std2**2) / (n1 + n2 - 2))
    
    # Cohen's d
    d = (mean1 - mean2) / pooled_std
    return d

# Calculate Effect Size
print("effect size \n", cohens_d(before_mean, after_mean))

김충연, paired t test
mean_before :
  1.0132228626948176e-09
mean_after : 
 3.4052216727259825e-09
KS Statistic: 
 0.49999937776854453
KS p-value: 
 0.0
t_statistics : 
 -31.171095433669432
p-value: 
 0.000000000000000000000000000000
effect size 
 -0.05200208890510275


# Analysis across Band

## Paired t-test with Cohen's D test

In [8]:
freq_ranges = {
    'Delta': (1, 4),
    'Theta': (4, 8),
    'Alpha': (8, 13),
    'Beta': (13, 30),
}

# Averaging by band and timing
for band, (fmin, fmax) in freq_ranges.items():
    tfr_band_before = tfr.copy().crop(tmin=0, tmax=3600, fmin=fmin, fmax=fmax, include_tmax=False)
    tfr_band_after = tfr.copy().crop(tmin=3600, tmax=7199, fmin=fmin, fmax=fmax, include_tmax=False)

    globals()[f'tfr_{band}_before'] = tfr_band_before
    globals()[f'tfr_{band}_after'] = tfr_band_after

    globals()[f'{band}_before_mean'] = tfr_mean(tfr_band_before)
    globals()[f'{band}_after_mean'] = tfr_mean(tfr_band_after)

# Make length same    
for band in freq_ranges.keys():
    before_mean = globals()[f'{band}_before_mean']
    after_mean = globals()[f'{band}_after_mean']

    if len(before_mean) > len(after_mean):
        globals()[f'{band}_before_mean'] = before_mean[:len(after_mean)]
    elif len(before_mean) < len(after_mean):
        globals()[f'{band}_after_mean'] = after_mean[:len(before_mean)]
    else:
        print(f"{band} 대역의 before와 after 길이가 이미 같습니다.")

    print(f"{band} 대역 처리 완료: before 길이 = {len(globals()[f'{band}_before_mean'])}, after 길이 = {len(globals()[f'{band}_after_mean'])}")

# Removing large artifact by band
def remove_artifacts(before_mean, after_mean, large_artifact):
    before_idx = set()
    after_idx = set()

    for t_start, t_end in large_artifact:
        if t_start < 3600 and t_end < 3600:
            before_idx.update(range(t_start*200, t_end*200))
            after_idx.update(range(t_start*200, t_end*200))
        elif t_start < 3600 and t_end >= 3600:
            before_idx.update(range(t_start*200, 3600*200))
            after_idx.update(range(t_start*200, 3600*200))
            after_idx.update(range(0, (t_end-3600)*200))
            before_idx.update(range(0, (t_end-3600)*200))
        else:
            after_idx.update(range((t_start-3600)*200, (t_end-3600)*200))
            before_idx.update(range((t_start-3600)*200, (t_end-3600)*200))

    before_idx = sorted(before_idx)
    after_idx = sorted(after_idx)

    before_mean_clean = np.delete(before_mean, before_idx)
    after_mean_clean = np.delete(after_mean, after_idx)

    return before_mean_clean, after_mean_clean

# 주파수 대역별로 artifact 제거
for band in freq_ranges.keys():
    before_mean = globals()[f'{band}_before_mean']
    after_mean = globals()[f'{band}_after_mean']

    before_mean_clean, after_mean_clean = remove_artifacts(before_mean, after_mean, large_artifact)

    globals()[f'{band}_before_mean_clean'] = before_mean_clean
    globals()[f'{band}_after_mean_clean'] = after_mean_clean

    print(f"{band} 대역 처리 완료:")
    print(f"  Before: 원본 길이 = {len(before_mean)}, 정제 후 길이 = {len(before_mean_clean)}")
    print(f"  After: 원본 길이 = {len(after_mean)}, 정제 후 길이 = {len(after_mean_clean)}")
    print(f"  정제 후 Before와 After 길이 일치: {len(before_mean_clean) == len(after_mean_clean)}")

print(f"\n {name}, paired t test by band")
def run_tests(before_mean, after_mean, band_name):
    print(f"\n=== {band_name} 대역 분석 결과 ===")
    
    # precision
    getcontext().prec = 30

    # mean of before vs after
    print(f"mean_before: \n {np.mean(before_mean)}")
    print(f"mean_after: \n {np.mean(after_mean)}")

    # Kolmogorov-Smirnov test
    difference = before_mean - after_mean
    statistic, p_value = stats.kstest(difference, stats.norm.cdf)
    print(f"KS Statistic: \n {statistic}")
    print(f"KS p-value: \n {p_value}")

    # t-test
    t_stat, p_value = stats.ttest_rel(before_mean, after_mean)

    # print to decimal object
    p_value_decimal = Decimal(p_value)
    print(f"t_statistics: \n {t_stat}")
    print(f"p-value: \n {p_value_decimal:.30f}")

    # Define Cohens'D 
    def cohens_d(group1, group2):
        mean1, mean2 = np.mean(group1), np.mean(group2)
        std1, std2 = np.std(group1, ddof=1), np.std(group2, ddof=1)
        n1, n2 = len(group1), len(group2)
        pooled_std = np.sqrt(((n1 - 1) * std1**2 + (n2 - 1) * std2**2) / (n1 + n2 - 2))
        d = (mean1 - mean2) / pooled_std
        return d

    # Calculate Effect Size
    effect_size = cohens_d(before_mean, after_mean)
    print(f"Effect size: \n {effect_size}")

# 각 주파수 대역별로 테스트 실행
for band in freq_ranges.keys():
    before_mean = globals()[f'{band}_before_mean_clean']
    after_mean = globals()[f'{band}_after_mean_clean']
    run_tests(before_mean, after_mean, band)

Delta 대역 처리 완료: before 길이 = 719800, after 길이 = 719800
Theta 대역 처리 완료: before 길이 = 719800, after 길이 = 719800
Alpha 대역 처리 완료: before 길이 = 719800, after 길이 = 719800
Beta 대역 처리 완료: before 길이 = 719800, after 길이 = 719800
Delta 대역 처리 완료:
  Before: 원본 길이 = 719800, 정제 후 길이 = 691200
  After: 원본 길이 = 719800, 정제 후 길이 = 691200
  정제 후 Before와 After 길이 일치: True
Theta 대역 처리 완료:
  Before: 원본 길이 = 719800, 정제 후 길이 = 691200
  After: 원본 길이 = 719800, 정제 후 길이 = 691200
  정제 후 Before와 After 길이 일치: True
Alpha 대역 처리 완료:
  Before: 원본 길이 = 719800, 정제 후 길이 = 691200
  After: 원본 길이 = 719800, 정제 후 길이 = 691200
  정제 후 Before와 After 길이 일치: True
Beta 대역 처리 완료:
  Before: 원본 길이 = 719800, 정제 후 길이 = 691200
  After: 원본 길이 = 719800, 정제 후 길이 = 691200
  정제 후 Before와 After 길이 일치: True

 김충연, paired t test by band

=== Delta 대역 분석 결과 ===
mean_before: 
 5.020405093624575e-09
mean_after: 
 1.988587588457064e-08
KS Statistic: 
 0.4999986940359352
KS p-value: 
 0.0
t_statistics: 
 -35.82919108329609
p-value: 
 0.00000000000000000000000

## RM ANOVA

In [ ]:
for band in freq_ranges.keys():
    before_mean = globals()[f'{band}_before_mean_clean']
    after_mean = globals()[f'{band}_after_mean_clean']
    
    before_mean_mean = np.mean(before_mean)
    after_mean_mean = np.mean(after_mean)
    
    globals()[f'{band.lower()}_before_mean_mean'] = before_mean_mean
    globals()[f'{band.lower()}_after_mean_mean'] = after_mean_mean
    
    print(f"\n=== {band} 대역 평균 ===")
    print(f"{band} Before Mean of Mean: {before_mean_mean}")
    print(f"{band} After Mean of Mean: {after_mean_mean}")

In [ ]:
data = pd.DataFrame({
    'Band': ['Delta', 'Delta', 'Theta', 'Theta', 'Alpha', 'Alpha', 'Beta', 'Beta'],
    'Time': ['Before', 'After', 'Before', 'After', 'Before', 'After', 'Before', 'After'],
    'Power': [delta_before_mean_mean, delta_after_mean_mean, theta_before_mean_mean, theta_after_mean_mean, 
              alpha_before_mean_mean, alpha_after_mean_mean, beta_before_mean_mean, beta_after_mean_mean]
})

In [ ]:
import statsmodels.api as sm
from statsmodels.formula.api import ols
from statsmodels.stats.anova import AnovaRM

# 반복측정 ANOVA 모델 설정
aovrm = AnovaRM(data, 'Power', 'Band', within=['Time'])
res = aovrm.fit()

# 결과 출력
print(res)